In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Test Spark").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [5]:
n_v = sc.parallelize([1,2,3,4])
n_v.map(lambda x:x*x*x).collect()

[1, 8, 27, 64]

In [6]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

In [7]:
data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')

In [8]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [9]:
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [11]:
predictions = model.predict(testData.map(lambda x: x.features))

In [12]:
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)

In [13]:
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())

SyntaxError: invalid syntax (Temp/ipykernel_2876/3207948612.py, line 1)

In [ ]:
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())